In [41]:
import numpy as np
import math

In [42]:
%run LanguageModel.py

In [43]:
%run DataLoader.py

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [45]:
train_dl = DataLoader('train', ('de', 'en'), device = device)

In [46]:
val_dl = DataLoader('dev', ('de', 'en'), train_dl.languageModels, device = device)

In [47]:
%run seq2seq.py

In [48]:
hidden_size = 256
lm1 = train_dl.languageModels[train_dl.languages[0]]
lm2 = train_dl.languageModels[train_dl.languages[1]]
s2s = seq2seq(lm1.n_tokens, hidden_size, lm2.n_tokens, device)

In [49]:
def loss_metric(input, output, ground_truth, nll):
    return nll / ground_truth[0].size(0)

def perplexity(input, output, ground_truth, nll):
    #print(ground_truth[0].size(0), nll)
    nll /= ground_truth[0].size(0)
    return math.exp(nll)

In [50]:
%run ScorePrinter.py

In [51]:
def trainIters(epochs, print_every=1000, plot_every=100):
    n_iters = len(train_dl)
    score_printer = ScorePrinter("Training", [('NLL', loss_metric),('Perplexity', perplexity)])
    
    for epoch in range(1, epochs+1):
        score_printer.startEpoch(epoch)
        idx_permutation = np.random.permutation(len(train_dl))[:5]
        for i, index in enumerate(idx_permutation):
            input_tensor, target_tensor = train_dl.tensorsFromPos(index)

            loss, output_sentence = s2s.train(input_tensor, target_tensor)
            score_printer.update(input_tensor, target_tensor, output_sentence, loss)
            
            if (i + 1) % print_every == 0:
                score_printer.printAvg(print_every)
                validate(print_every)
        
        score_printer.endEpoch(epoch)
        score_printer.printAvg(len(train_dl))
        validate()

In [52]:
def validate(n = None):
    score_printer = ScorePrinter("Validation", [('NLL', loss_metric),('Perplexity', perplexity)])
    n = n or len(val_dl)
    idx_permutation_val = np.random.permutation(len(val_dl))[:n]
    for j, val_index in enumerate(idx_permutation_val):
        input_tensor_val, target_tensor_val = val_dl.tensorsFromPos(val_index)
        loss, output_sentence = s2s.evaluate(input_tensor_val, target_tensor_val)
        score_printer.update(input_tensor_val, target_tensor_val, output_sentence, loss)
    score_printer.printAvg(showCount = False)

In [ ]:
trainIters(50, print_every=2)